In [ ]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [1]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO, DelimitedFiles, Tables, DataFrames, Statistics
#using HypothesisTests #contient ChisqTest

plotly()

Plots.PlotlyBackend()

### Constantes

In [2]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Fit global sur l'ensemble des lenslets:

On definit les parametres qui servent a creer le modele:
a l'amplitude maximale des spots gaussien,
fwhm la largeur a mi hauteur des spots,
cx les coefficients polynomiaux le long de l'axe x,
cy les coefficients polynomiaux le long de l'axe y

### Exemple

In [3]:
#Bbox of the lenslet: box autour d'un set de 3 gaussiennes
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);
a0 = [300.0 , 400.0, 500.0];
fwhm0 = [2.0, 2.0 , 2.0];
#affichage d'un exemple de trois gaussiennes avec la bbox sur les donnees reelles:
exemple = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")
data= read(exemple[1]);
exempleData = data[1013:1017 , 1050:1090]
heatmap(exempleData, title = "Lenslet correspondant a la bbox initiale (exemple)", aspect_ratio = 1)

<!DOCTYPE html>
 
 
 Plots.jl

On peut deduire de cette image la taille de la bounding box d'une lenslet: hauteur 4 pixels, largeur 40 pixels.
Il manque les coeff C0 pour pouvoir fiter les gaussiennes, qu'on recupere dans des fichiers txt

### 1 lecture des fichiers txt

On veut recuperer en sortie de fonction la premiere colonne (c0, coef a l'ordre 0, soit le centre de la lenslet).

x(λ) = cx0 + cx1*((λ-λ0)/λ0) + cx2*((λ-λ0)/λ0)²

y(λ) = cy0 + cy1*((λ-λ0)/λ0) + cy2*((λ-λ0)/λ0)²

On donne ici λ en metre, dans le code Matlab λ est en micronmetre. Il faut donc normaliser et convertir les coefficients c1 et c2. 

In [4]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1];
cx1 = coeffx[:,2]*λ0*1e6;
cx2 = coeffx[:,3]*λ0^2*1e12;

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1];
cy1 = coeffy[:,2]*λ0*1e6;
cy2 = coeffy[:,3]*λ0^2*1e12;

### Histogrammes

In [ ]:
h1 = histogram(cx1, title = "cx1");
h2 = histogram(cx2, title = "cx2");
plot(h1, h2, legend = false)

In [ ]:
h3 = histogram(cy1, title = "cy1");
h4 = histogram(cy2, title = "cy2");
plot(h3, h4, legend = false)

### 2 ouvrir et trouver positions dans l'image brute

1/ IFS_sim_wave.fits: on ouvre le modele, qui a peu de BP, donc les fit des lenslets seront bons. f5 = FITS("/home/user/Pictures/IFS_sim_wave.fits")

2/ IFS_calib_wave_corrected: "vrai" fichier de depart (a comparer avec 1/ eventuellement)

In [5]:
laserData = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits", "r");

In [6]:
#Read the data or a subset thereof from disk, and save it in a pre-allocated output array A.
A =zeros(2048,2048); #pre-allocate matrix A
read!(laserData[1], A); #data

In [7]:
close(laserData);

In [8]:
#Constantes
nb_fit = 5 #test sur quelques lenslets

ainit = [300. , 400. , 500.];
fwhminit = [2.0, 2.0 , 2.0];
laser =  LaserModel(λlaser,ainit,fwhminit);
Cinit = zeros(2,laser.nλ);

#BoundingBox
largeur = 4;
hauteur = 40;
dxmin = round(Int, largeur/2);
dxmax = round(Int, largeur/2);
dymin = round(Int, hauteur/2);
dymax = round(Int, hauteur/2);
mcx1 = median(cx1);
mcy1 = median(cy1);
mcx2 = median(cx2);
mcy2 = median(cy2);

N = length(cx0) #Nombre de lenslet (18908)

valid = (round.(cx0 .- dxmin).>0) .&  (round.(cx0 .+ dxmax).<2048) .&  (round.(cy0 .- dymin).>0) .&  (round.(cy0 .+ dymax).<2048);

In [9]:
#Initialiser tableaux de resultats avec possibilite d'avoir valeurs "missing"
atab = Array{Union{Float64,Missing}}(missing,3,N); #zeros(3,N);
fwhmtab = Array{Union{Float64,Missing}}(missing,3,N); #zeros(3,N);
ctab = Array{Union{Float64,Missing}}(missing,2,3,N); #zeros(2,3,N);
lmodtab = Array{Union{LensletModel,Missing}}(missing,N); #Array{LensletModel}(undef,N);

In [ ]:
atab

### Lenslet aleatoire sans prise en compte des pixels defectueux
xmin = cx0[i,1]-largeur/2 lenslet, ymin = cy0[i,1]-hauteur/2 lenslet

xmax = cx0[i,1]+largeur/2 lenslet, ymax = cy0[i,1]+hauteur/2 lenslet

Remarque: centre de l'image 0 = 1025x1025 pixel, donc il faut ajouter 1025 a xmin et xmax, ymin et ymax 

In [10]:
i = 5
#Initialisation 
cx0i = cx0[i,1] .+1025; #centre de la lenslet
cy0i = cy0[i,1] .+1025;

xmin=cx0i -dxmin;
xmax=cx0i +dxmax;
ymin=cy0i -dymin;
ymax=cy0i +dymax; 

bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax)); 

lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet

Cinit = zeros(2,laser.nλ);
Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];

#Fit du polynome pour chaque lenslet: optimisation
ldata = view(A, bbox);
lkl = LikelihoodIFS(lmodtab[i], laser, ldata);
cost(x::Vector{Float64}) = lkl(x)::Float64;
xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

#Sauvegarde des valeurs des parametres du modele dans des vecteurs
#ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
atab[:,i] = aopt;
fwhmtab[:,i] = fwhmopt;
ctab[:,:,i]  = copt;


# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.1644274376639216E+06  9.10E+05  0.00E+00
    50     57      0    4.3035758015814552E+05  2.10E+04  1.00E+00
   100    111      0    1.2188155732275905E+05  4.33E+03  1.00E+00
   150    166      0    1.2057711009030815E+05  4.41E+03  1.00E+00
   200    223      0    1.1985169339853752E+05  9.34E+02  1.00E+00
   250    277      0    1.1961584521382426E+05  2.90E+03  1.00E+00
   300    330      0    1.1871550754739335E+05  6.72E+03  1.00E+00
   350    384      0    1.1561182719291747E+05  1.25E+04  1.00E+00
   400    439      0    1.1380453609711648E+05  6.35E+02  1.00E+00


In [11]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [25]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), title = "lenslet 5", xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis,  transpose(limageinit), title = "Initialisation", xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [14]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis,  transpose(limageopt), title = "Point d'arrivee de l'optimiseur", xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus sans precision", xlabel = "x", ylabel = "y", aspect_ratio = 1 )

<!DOCTYPE html>
 
 
 Plots.jl

In [16]:
@show(ainit,aopt,);

@show(fwhminit,fwhmopt);

@show(Cinit,copt);

ainit = [300.0, 400.0, 500.0]
aopt = [978.4291799788813, 677.3126304628319, 619.4889053279672]
fwhminit = [2.0, 2.0, 2.0]
fwhmopt = [2.254523487266258, 2.3741053651236075, 2.4887114132663366]
Cinit = [999.5222834913245 -0.6843665411452149 -0.41446582217274436; 1073.4407472368255 102.60070991107642 -68.43665668411037]
copt = [999.517706523665 -0.639341072434858 -1.803139952491615; 1073.4624158780084 102.87429999286442 -68.06074633439525]


### Lenslet aleatoire avec prise en compte des pixels defectueux

In [17]:
badpix = Float64.(read(FITS("/home/user/stage/HR_4796-HD_95086/IFS_BP_corrected.fits")[1]));

In [18]:
i = 5
#Initialisation 
cx0i = cx0[i,1] .+1025; #centre de la lenslet
cy0i = cy0[i,1] .+1025;

xmin=cx0i .-dxmin;
xmax=cx0i .+dxmax;
ymin=cy0i .-dymin;
ymax=cy0i .+dymax; 

bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax)); 

lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet

Cinit = zeros(2,laser.nλ);
Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];



#Fit du polynome pour chaque lenslet: optimisation
ldata = view(A, bbox);
badpixview = view(badpix,bbox)
lkl = LikelihoodIFS(lmodtab[i], laser, ldata, badpixview);#rajouter la precision avec la carte des BP: view(precision,lenslet.bbox)
cost(x::Vector{Float64}) = lkl(x)::Float64;
xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

#Sauvegarde des valeurs des parametres du modele dans des vecteurs
#ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
atab[:,i] = aopt;
fwhmtab[:,i] = fwhmopt;
ctab[:,:,i]  = copt;

# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.0589786643102935E+06  8.41E+05  0.00E+00
    50     56      0    4.6021526531514863E+05  1.45E+04  1.00E+00
   100    109      0    3.8034864741808351E+05  1.19E+04  1.00E+00
   150    163      0    3.0769962388261687E+05  1.34E+04  1.00E+00
   200    215      0    1.1667850152955098E+05  4.08E+03  1.00E+00
   250    272      0    1.1169890769763068E+05  2.48E+03  1.00E+00
   300    323      0    1.1120502565586525E+05  5.62E+03  1.00E+00
   350    376      0    1.0603637029609396E+05  1.73E+03  1.00E+00
   400    431      0    1.0462109345124994E+05  7.61E+03  3.17E-01
   450    485      0    1.0442337761108842E+05  2.78E+03  1.00E+00


In [19]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [26]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), title = "lenslet 5", xlabel = "x", ylabel = "y", aspect_ratio = 1,  legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [21]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis, transpose(limageinit), title = "Initialisation", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [27]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis,  transpose(limageopt), title = "Point d'arrivee de l'optimiseur avec precision", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [23]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus avec precision", xlabel = "x", ylabel = "y", aspect_ratio = 1)

<!DOCTYPE html>
 
 
 Plots.jl

In [24]:
@show(ainit,aopt,);

@show(fwhminit,fwhmopt);

@show(Cinit,copt);

ainit = [300.0, 400.0, 500.0]
aopt = [969.5739920310513, 651.8675476556057, 603.0197731104628]
fwhminit = [2.0, 2.0, 2.0]
fwhmopt = [2.291736241188877, 2.433464793747011, 2.52448834485326]
Cinit = [999.5222834913245 -0.6843665411452149 -0.41446582217274436; 1073.4407472368255 102.60070991107642 -68.43665668411037]
copt = [999.5447833129792 -0.585168064047276 -3.632122262562186; 1073.4578680146467 102.88050651569068 -68.03905967270322]


### Ensemble des lenslet

In [221]:
for i in collect(1:N)[valid]
    #Initialisation
    cx0i = cx0[i,1] .+1025; #centre de la lenslet
    cy0i = cy0[i,1] .+1025;

    xmin=cx0i .-dxmin;
    xmax=cx0i .+dxmax;
    ymin=cy0i .-dymin;
    ymax=cy0i .+dymax; 

    bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));
    
    lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet
     
    Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];
    #Cinit[1:2,1:3] = [ [cy0[i,1] mcy1 mcy2]; [cx0[i,1] mcx1 mcx2] ];
    
    #Fit du polynome pour chaque lenslet: optimisation
    ldata = view(A, bbox);    #lenslet laser data
    lkl = LikelihoodIFS(lmodtab[i], laser, ldata);#rajouter carte BP
    cost(x::Vector{Float64}) = lkl(x);
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

    #Sauvegarde des valeurs des parametres du modele dans des vecteurs
    #ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
    (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    atab[:,i] = aopt
    fwhmtab[:,i] = fwhmopt
    ctab[:,:,i]  = copt
        
    print(i)
end


# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    4.9909879451595671E+07  8.37E+05  0.00E+00
    50     63      0    4.5118983591263406E+07  1.36E+05  1.00E+00
   100    168      0    5.2289793556406982E+06  1.14E+07  1.00E+00
   150    241      0    5.2079381914904006E+06  1.21E+05  1.00E+00
   200    307      0    5.2011522369226478E+06  2.70E+05  1.00E+00
   250    377      0    4.9829005717716794E+06  6.88E+06  1.00E+00
   300    445      0    4.9720097134072594E+06  8.13E+05  1.00E+00
# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.2964650834791423E+06  9.17E+05  0.00E+00
    50     55      0    9.0049557173353049E+05  4.26E+04  1.00E+00
   100    107      0    5.5368112626312650E+05  9.69E+04  1.00E+00
   150    162      0    2.5517524279089281E+05  3.19E+04  1.00E+00

LoadError: [91mInterruptException:[39m

In [15]:
collect(1:N)[valid] #indice des lenslet valides

4463-element Array{Int64,1}:
     2
     8
     9
    10
    11
    20
    21
    22
    23
    25
    26
    38
    39
     ⋮
 18392
 18394
 18395
 18396
 18397
 18465
 18467
 18468
 18469
 18470
 18532
 18533

In [257]:
for i in 1:200
    #Initialisation
    cx0i = cx0[i,1] .+1025;
    cy0i = cy0[i,1] .+1025;

    xmin=cx0i .-dxmin;
    xmax=cx0i .+dxmax;
    ymin=cy0i .-dymin;
    ymax=cy0i .+dymax; 

    bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));
    
    lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet
     
    Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];
    
    #Fit du polynome pour chaque lenslet: optimisation
    ldata = view(A, bbox);    #lenslet laser data
    lkl = LikelihoodIFS(lmodtab[i], laser, ldata);#rajouter carte BP
    cost(x::Vector{Float64}) = lkl(x);
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

    #Sauvegarde des valeurs des parametres du modele dans des vecteurs
    (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    atab[:,i] = aopt
    fwhmtab[:,i] = fwhmopt
    ctab[:,:,i]  = copt
    
    print(i)
end


# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.9260141180574277E+06  7.88E+05  0.00E+00
    50     53      0    1.6621929918916285E+05  1.17E+04  1.00E+00
   100    108      0    1.4572065492935278E+05  1.08E+04  1.00E+00
   150    160      0    1.3326275169128668E+05  5.29E+03  1.00E+00
   200    213      0    1.3232252089663132E+05  2.68E+03  1.00E+00
   250    266      0    1.3151645189578843E+05  2.78E+03  1.00E+00
   300    324      0    1.3098417397569506E+05  1.59E+03  5.00E+00
   350    382      0    1.3075350305851088E+05  2.51E+03  1.00E+00
   400    435      0    1.1432472664530615E+05  1.35E+04  1.00E+00
   450    489      0    1.1124956492253488E+05  2.22E+03  1.00E+00
1# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    4.9909879451595671E+07  8.37E+05  0.00E+0

   250    268      0    1.2601651111106580E+05  2.01E+03  2.74E-01
   300    321      0    1.2520811877461850E+05  4.51E+03  1.00E+00
   350    371      0    1.2439585377017537E+05  4.91E+03  1.00E+00
   400    423      0    1.2404746496224002E+05  1.24E+03  1.00E+00
   450    477      0    1.2389733124987749E+05  5.53E+03  1.00E+00
12# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    6.7807137748555064E+07  7.93E+05  0.00E+00
    50     77      0    6.2061350213310711E+07  8.12E+05  4.18E-01
   100    145      0    5.4561206085548475E+07  4.70E+06  9.82E-02
   150    210      0    3.5290819499754556E+06  1.23E+05  1.00E+00
13# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.0817729369231018E+06  8.38E+05  0.00E+00
    50     54      0    7.1924576334773423E+05  1.51E+05  1.00

   100    113      0    1.2018989711524665E+06  3.30E+04  1.00E+00
   150    167      0    1.1808822791828171E+06  2.17E+04  1.00E+00
   200    223      0    9.6774825701724796E+05  1.83E+04  1.00E+00
   250    280      0    4.6069653205310274E+05  6.59E+04  1.00E+00
   300    335      0    4.1363812868859159E+05  4.04E+03  1.00E+00
   350    390      0    4.1339578680582857E+05  5.10E+03  1.72E-01
   400    443      0    4.1327876278040814E+05  1.13E+04  1.00E+00
   450    498      0    3.7026047231749725E+05  5.82E+03  1.00E+00
25# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.9787656722256530E+06  7.87E+05  0.00E+00
    50     54      0    8.6170451279672363E+05  2.98E+04  1.00E+00
   100    106      0    6.1544028459783422E+05  2.07E+04  1.65E-01
   150    163      0    4.4010505208549538E+05  3.71E+04  4.18E-01
   200    220      0    3.6468960733515286E+05  1.09E+05  4.24

   100    110      0    2.6235764028682129E+05  5.34E+03  1.00E+00
   150    164      0    2.5140433538027335E+05  1.04E+04  1.00E+00
   200    217      0    1.7099147416101577E+05  9.92E+04  1.00E+00
   250    271      0    1.0022939625772348E+05  7.25E+02  1.00E+00
   300    327      0    1.0019647551038660E+05  1.43E+02  1.00E+00
   350    380      0    1.0019446020657867E+05  1.19E+02  1.00E+00
   400    432      0    1.0016828411285859E+05  2.53E+03  1.00E+00
   450    485      0    1.0012096135441249E+05  4.09E+03  1.00E+00
38# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.2681833945352794E+06  8.56E+05  0.00E+00
    50     53      0    7.0106417939925287E+05  1.09E+05  4.23E-01
   100    107      0    4.0704515341202513E+05  1.71E+04  1.00E+00
   150    159      0    1.2921668724958986E+05  7.28E+03  1.00E+00
   200    212      0    1.2526224008075042E+05  4.64E+03  1.00

   100    152      0    7.2068476820599452E+07  1.64E+06  1.00E+00
   150    225      0    9.5478804968201574E+06  3.49E+05  1.00E+00
49# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.7407332614400536E+08  9.73E+05  0.00E+00
    50     77      0    2.2247833467255357E+08  1.62E+06  1.00E+00
   100    134      0    2.1472667887925231E+08  3.76E+06  1.00E+00
   150    191      0    1.9395958179918551E+08  5.24E+06  1.00E+00
   200    250      0    5.4658693418335058E+06  2.09E+06  1.00E+00
   250    313      0    4.3973537385317544E+06  1.05E+06  1.00E+00
   300    376      0    4.3902878999771914E+06  3.55E+06  1.00E+00
   350    438      0    4.3792348034144156E+06  2.02E+05  1.00E+00
50# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.3865654307477152E+08  3.05E+06  0.00

...

   100    108      0    5.8174228209635604E+05  1.30E+05  1.00E+00
   150    162      0    4.1493128886806691E+05  1.19E+05  1.00E+00
   200    218      0    1.2488910160339759E+05  9.31E+03  1.00E+00
   250    272      0    1.2323482706795863E+05  6.17E+03  1.00E+00
   300    325      0    1.2250788705129598E+05  3.47E+03  1.00E+00
   350    380      0    1.2238005051720375E+05  1.22E+03  1.00E+00
   400    434      0    1.2220803995696090E+05  8.39E+03  1.00E+00
   450    489      0    1.1886389267181836E+05  2.52E+03  1.00E+00
146# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.5018685383998230E+06  9.61E+05  0.00E+00
    50     60      0    7.0629442116144509E+05  4.08E+04  1.00E+00
   100    114      0    1.2615813750870281E+05  2.55E+05  8.76E-02
   150    169      0    9.7574423917000866E+04  1.60E+03  1.00E+00
   200    223      0    9.7413303972149894E+04  5.52E+02  1.0

   200    221      0    2.2620268428314524E+05  7.95E+02  1.00E+00
   250    275      0    2.2599217223180647E+05  4.31E+02  1.00E+00
   300    329      0    2.2598239426807701E+05  2.71E+02  3.24E-01
   350    382      0    2.2597207666520929E+05  2.08E+02  1.00E+00
   400    434      0    2.2596338337732846E+05  3.63E+02  1.00E+00
157# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.9917747954661411E+06  8.44E+05  0.00E+00
    50     56      0    7.3966422595487186E+05  6.74E+04  1.00E+00
   100    109      0    3.5327756407352845E+05  6.50E+04  1.00E+00
   150    163      0    3.3500244635791692E+05  2.35E+04  1.00E+00
   200    215      0    1.9227029830805026E+05  9.73E+03  1.00E+00
   250    271      0    1.2456143808972024E+05  2.42E+03  1.00E+00
   300    324      0    1.2379052161935302E+05  1.61E+03  1.00E+00
   350    382      0    1.2367474497610588E+05  2.31E+03  1.0

   200    215      0    1.1899749693623422E+05  1.62E+04  1.00E+00
   250    269      0    1.1488324600935938E+05  1.42E+04  1.00E+00
   300    322      0    1.1329905488564566E+05  2.45E+03  1.00E+00
   350    377      0    1.1143077340098684E+05  1.67E+04  1.00E+00
   400    432      0    1.0505967498906895E+05  3.05E+04  3.62E-01
   450    489      0    1.0453573921566435E+05  3.96E+03  1.00E+00
168# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    3.0164709703776040E+06  1.07E+06  0.00E+00
    50     55      0    5.3577746732242708E+05  6.88E+04  1.00E+00
   100    110      0    4.2337630098701827E+05  8.69E+03  1.00E+00
   150    165      0    2.5193027328808428E+05  1.35E+05  1.00E+00
   200    222      0    1.1510760862162769E+05  1.92E+04  1.00E+00
   250    276      0    9.5948572947495719E+04  2.77E+04  1.00E+00
   300    330      0    9.4323581499993394E+04  8.55E+02  1.0

   100    119      0    9.6944785575963203E+06  8.60E+03  3.59E-01
   150    172      0    9.6501896126723755E+06  1.96E+04  1.00E+00
   200    227      0    9.6439596365703400E+06  9.65E+03  2.31E-01
   250    283      0    9.6146950465444233E+06  1.12E+04  1.00E+00
   300    335      0    9.6013041965874191E+06  1.25E+04  1.00E+00
   350    390      0    9.5864220997407530E+06  9.77E+03  1.00E+00
   400    444      0    9.5709992815438528E+06  7.34E+03  1.00E+00
   450    497      0    9.5690739627407361E+06  4.10E+03  1.00E+00
181# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    1.2211963466863941E+08  2.15E+06  0.00E+00
    50     67      0    6.2607024320607102E+06  4.00E+06  1.00E+00
182# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    3.5907784245528616E+07  7.65E+05  0.

   200    219      0    1.1021228408571046E+05  2.72E+03  1.00E+00
   250    274      0    1.1006349332839747E+05  7.66E+02  1.00E+00
   300    327      0    1.0925446552584015E+05  8.02E+03  1.00E+00
   350    378      0    1.0582947098494018E+05  1.74E+04  1.00E+00
   400    429      0    1.0333395753224993E+05  1.93E+04  1.00E+00
   450    483      0    1.0029194742850523E+05  2.67E+02  1.00E+00
192# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    2.1829071806599512E+06  8.09E+05  0.00E+00
    50     55      0    3.2668589150177094E+05  1.03E+04  1.00E+00
   100    106      0    2.4030987376466341E+05  3.05E+04  1.00E+00
   150    160      0    1.6177184792250255E+05  2.47E+04  1.00E+00
   200    214      0    1.5232671451524968E+05  4.29E+04  3.11E-01
   250    271      0    1.3976892081234598E+05  1.87E+04  1.00E+00
   300    323      0    1.3509679971069130E+05  1.41E+04  1.0

In [258]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [259]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [260]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis,  transpose(limageinit), title = "Initialisation", xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [261]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis,  transpose(limageopt), title = "Point d'arrivee de l'optimiseur", xlabel = "x", ylabel = "y", aspect_ratio = 1, legend = :bottomright)

<!DOCTYPE html>
 
 
 Plots.jl

In [262]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus sans precision", xlabel = "x", ylabel = "y", aspect_ratio = 1)

<!DOCTYPE html>
 
 
 Plots.jl

In [238]:
@show(ainit,aopt);

@show(fwhminit,fwhmopt);

@show(Cinit,copt);

ainit = [300.0, 400.0, 500.0]
aopt = [1073.48622631668, 719.517227254495, 355.51815367423245]
fwhminit = [2.0, 2.0, 2.0]
fwhmopt = [2.2999991296742572, 2.3674313832176157, -2.259794852447017]
Cinit = [1070.6427114255894 -0.6843665411452149 -0.41446582217274436; 1238.1295158479923 102.60070991107642 -68.43665668411037]
copt = [1070.5798169492061 -0.8443396462228552 3.092728971871351; 1238.0634595845397 102.07321851634873 -65.3939631623928]


In [239]:
atab

3×18908 Array{Union{Missing, Float64},2}:
 966.684  19970.7    1180.74   453.319  …  missing  missing  missing  missing
 601.274  64039.2     526.4    442.8       missing  missing  missing  missing
 437.477    489.354   288.615  485.973     missing  missing  missing  missing

In [240]:
DataSave = FITS("newfile.fits", "w");
write(DataSave, atab)  # Write a new image extension with the data
close(DataSave)

LoadError: [91mMethodError: no method matching write(::FITS, ::Array{Union{Missing, Float64},2})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  write(::FITS, [91m::Array{String,1}[39m, [91m::Array{T,1} where T[39m; units, header, hdutype, name, ver, varcols) at /home/user/.julia/packages/FITSIO/dBcm0/src/table.jl:358[39m
[91m[0m  write(::FITS, [91m::StridedArray{var"#s41", N} where N where var"#s41"<:Real[39m; header, name, ver) at /home/user/.julia/packages/FITSIO/dBcm0/src/image.jl:336[39m
[91m[0m  write(::FITS, [91m::Dict{String,V} where V[39m; units, header, hdutype, name, ver, varcols) at /home/user/.julia/packages/FITSIO/dBcm0/src/table.jl:406[39m
[91m[0m  ...[39m

### 3 affichage

In [ ]:
i = 5
#Initialisation
#xmin = cx0[i,1]-largeur/2 lenslet, ymin = cy0[i,1]-hauteur/2 lenslet
#xmax = cx0[i,1]+largeur/2 lenslet, ymax = cy0[i,1]+hauteur/2 lenslet
#Remarque: centre de l'image 0 = 1025x1025 pixel, donc il faut ajouter 1025 a xmin et xmax  
xmin=(cx0[i,1] .-dxmin) .+1026;
xmax=(cx0[i,1] .+dxmax) .+1026;
ymin=(cy0[i,1] .-dymin) .+1024;
ymax=(cy0[i,1] .+dymax) .+1024;   
bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));
print(bbox.xmin)
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);
    
#Test avec donnees lamp pour verifier que la bbox est convenable: si spectre est complet ou decoupee ou pas sur quelques lenslet
#f6 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
plt = scatter(cx0[1:10,1], cx0[1:10,1])
#bbox = BoundingBox(xmin=1013, xmax=1017, ymin=1050, ymax=1088);
ldata = view(B,bbox); # ex leslet au centre de l'image [1012:1018, 1048:1092]
p1t = heatmap( y_axis, x_axis, ldata, title = "Exemple du spectre d'une lenslet au centre")


### Test avec donnees lamp
Permet de verifier si la bbox es convenable (si spectre est complet ou decoupee ou pas sur quelques lenslet)

In [ ]:
f6 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
B =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f6[1], B);
bbox = BoundingBox(xmin=1013, xmax=1017, ymin=1050, ymax=1088);
ldata = view(B,bbox); # ex lenslet au centre de l'image [1012:1018, 1048:1092]
p1 = heatmap(ldata, title = "Exemple du spectre d'une lenslet au centre")


f7 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
C =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f7[1], C);
bbox = BoundingBox(xmin=214, xmax=218, ymin=272, ymax=311);
ldata = view(C,bbox); 
p2 = heatmap(ldata, title = "Exemple 1 du spectre d'une lenslet en peripherie")


f8 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
D =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f7[1], D);
bbox = BoundingBox(xmin=1501, xmax=1505, ymin=1574, ymax=1613);
ldata = view(D,bbox); 
p3 = heatmap(ldata, title = "Exemple 2 du spectre d'une lenslet en peripherie")


plot(p1, p2, p3, legend = false, layout = (3, 1), aspect_ratio = 1)

On voit que le spectre n'est pas decoupe.

In [ ]:
#toute l'image
lamp = FITS("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/IFS_sim_spec.fits")
data = read(lamp[1]);
bbox = BoundingBox(1, 2048, 1, 2048);
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);
plt = scatter(cx0[:,1], cy0[:,1])
heatmap!(plt,x_axis, y_axis,  transpose(data), title = "IFS_sim_spec.fits")

In [ ]:
#test
plt = scatter(cx0[1:10,1] .+200, cy0[1:10,1] .+400)
heatmap!(plt,x_axis, y_axis,  transpose(data), title = "IFS_sim_spec.fits")

Methode 1: tracer des histogrammes de tous les coeff, visualisation des outliers. Ajustement d'une gaussienne, calcul de la moyenne de la variance, ce qui est superieur a 3σ est sorti.

Methode 2: test statistique par rapport au guess si methode 1 marche pas. Histogramme des valeurs de χ2

In [242]:
gr()
h5 = histogram(atab)
h6 = histogram(fwhmtab)
h7 = histogram(ctab)
plot(h5, h6, h7, legend = false)

LoadError: [91mMethodError: convert(::Type{Union{}}, ::Float64) is ambiguous. Candidates:[39m
[91m  convert(::Type{Union{}}, x) in Base at essentials.jl:169[39m
[91m  convert(::Type{T}, x::Number) where T<:Number in Base at number.jl:7[39m
[91m  convert(::Type{T}, arg) where T<:VecElement in Base at baseext.jl:8[39m
[91m  convert(::Type{T}, x::Number) where T<:AbstractChar in Base at char.jl:179[39m
[91m  convert(::Type{C}, c::Number) where C<:Colorant in ColorTypes at /home/user/.julia/packages/ColorTypes/7OlxI/src/conversions.jl:74[39m
[91m  convert(::Type{T}, A) where T<:LazyAlgebra.SparseOperators.SparseOperator in LazyAlgebra.SparseOperators at /home/user/.julia/packages/LazyAlgebra/ukdzB/src/sparse.jl:334[39m
[91mPossible fix, define[39m
[91m  convert(::Type{Union{}}, ::Number)[39m

In [241]:
#Statistics
@show std(aopt);
@show std(fwhmopt);
@show std(copt);

std(aopt) = 358.99571288022486
std(fwhmopt) = 2.652278608549964
std(copt) = 595.8410850164644


In [243]:
@show mean(var(aopt));
@show var(fwhmopt);
@show var(copt);

mean(var(aopt)) = 128877.92186638086
var(fwhmopt) = 7.034581817371735
var(copt) = 355026.59859359753


In [ ]:
#loi de dispersion
wxy0 = lmod.dmodel.(wavelengthrange);
(wx0,wy0) = collect.( zip(wxy0...));

In [ ]:
lmod.dmodel.(λlaser) 
#heatmap(ldata, title = "Fit laser") #1/ pour IFS_sim_wave.fits
heatmap(clamp.(ldata,0,1500), title = "Fit laser")
plot!(wy0,wx0; label="dispersion law")